In [2]:
import json
import os
import pandas as pd
from copy import deepcopy
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from collections import Counter
import numpy as np
from scipy import stats
import seaborn as sns 
from matplotlib.gridspec import GridSpec
from datetime import datetime
from scipy.stats import pearsonr
import statsmodels.api as sm
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import shap
from xgboost import XGBRegressor
from collections import defaultdict
from scipy.stats import ttest_rel
from scipy.stats import sem


In [3]:
def load_json(path):             # JSON 데이터 로드 함수
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

def save_json(path, data):
    with open(path, "w", encoding="utf-8") as f:  # 인코딩을 UTF-8로 지정
        json.dump(data, f, ensure_ascii=False, indent="\t")

In [4]:
#데이터 불러오기
raw_contents_info = pd.read_csv("military_data_62.csv")

우울 척도 점수 변화
--

* 우울 척도 변화 함수

In [5]:
# 사전 및 사후 CESD-10-D 점수를 추출하는 함수
def extract_pre_post_scores_from_csv(data):
    pre_scores = []
    post_scores = []
    
    # 'PRE_SCORE'와 'POST_SCORE'에서 점수를 추출
    for _, row in data.iterrows():
        try:
            # 사전 점수 추출
            pre_score = row["PRE_SCORE"]
            # 사후 점수 추출
            post_score = row["POST_SCORE"]
            # 점수를 리스트에 추가
            pre_scores.append(pre_score)
            post_scores.append(post_score)
        except KeyError:
            # 열 이름이 없거나 잘못된 경우 건너뜀
            continue
        except TypeError:
            # 값이 비어 있거나 잘못된 형식일 경우 건너뜀
            continue
    
    return pre_scores, post_scores


# 중복 제거 후 고유 사용자 수 계산 함수
def count_unique_users(data):
    unique_user_ids = data["순번"].unique()  # '순번' 기준으로 고유 사용자 ID 추출
    return len(unique_user_ids)



* 우울 척도 점수 차이 검정 함수

In [6]:
# 95% 신뢰구간 계산 함수
def calculate_confidence_interval(data, confidence=0.95):
    mean = np.mean(data)
    std_err = stats.sem(data)  # 표준 오차
    n = len(data)  # 샘플 크기
    margin = std_err * stats.t.ppf((1 + confidence) / 2, df=n-1)
    return mean - margin, mean + margin

# CESD-10-D 통계 분석 함수 (수정됨)
def calculate_statistics(pre_scores, post_scores):
    if len(pre_scores) != len(post_scores):
        raise ValueError("Pre-test and Post-test scores must have the same number of samples.")
    
    # 평균, 표준 편차, 표준오차평균 계산
    pre_mean = np.mean(pre_scores)
    pre_std = np.std(pre_scores, ddof=1)
    pre_sem = stats.sem(pre_scores)
    post_mean = np.mean(post_scores)
    post_std = np.std(post_scores, ddof=1)
    post_sem = stats.sem(post_scores)
    
    # 점수 변화 계산
    score_changes = np.array(post_scores) - np.array(pre_scores)
    change_mean = np.mean(score_changes)
    change_std = np.std(score_changes, ddof=1)
    
    # 95% 신뢰 구간 계산
    lower_ci, upper_ci = calculate_confidence_interval(score_changes)
    
    # t-검정 수행
    t_stat, p_value = stats.ttest_rel(pre_scores, post_scores)
    df = len(pre_scores) - 1  # 자유도
    
    # 결과 딕셔너리 반환
    return {
        "pre_mean": pre_mean,
        "pre_std": pre_std,
        "pre_sem": pre_sem,
        "post_mean": post_mean,
        "post_std": post_std,
        "post_sem": post_sem,
        "change_mean": change_mean,
        "change_std": change_std,
        "confidence_interval": (lower_ci, upper_ci),
        "t_value": t_stat,
        "p_value": p_value,
        "df": df
    }


* 우울 척도 점수 증가/감소/변화없음 분포 함수

In [7]:
# CESD-10-D 변화 분석 함수
def calculate_change_statistics(pre_scores, post_scores):
    if len(pre_scores) != len(post_scores):
        raise ValueError("Pre-test and Post-test scores must have the same number of samples.")
    
    # 점수 변화 계산
    score_changes = np.array(post_scores) - np.array(pre_scores)
    
    # 변화 유형 분류
    increase = score_changes[score_changes > 0]
    decrease = score_changes[score_changes < 0]
    no_change = score_changes[score_changes == 0]
    
    # 통계 계산 함수
    def calculate_stats(group):
        if len(group) == 0:  # 그룹이 비어있을 경우
            return {
                "count": 0,
                "mean": None,
                "std": None,
                "sem": None
            }
        return {
            "count": len(group),
            "mean": np.mean(group),
            "std": np.std(group, ddof=1),
            "sem": sem(group)
        }
    
    # 그룹별 통계 계산
    increase_stats = calculate_stats(increase)
    decrease_stats = calculate_stats(decrease)
    no_change_stats = calculate_stats(no_change)
    
    
    # 결과 딕셔너리 반환
    return {
        "increase": increase_stats,
        "decrease": decrease_stats,
        "no_change": no_change_stats
    }

* 뼈대 함수

In [8]:
def analyze_by_criteria(data, filter_column=None, filter_value=None):
    """
    특정 기준으로 데이터를 필터링한 후 우울 척도 점수 분석을 수행합니다.

    Args:
        data (pd.DataFrame): 전체 데이터프레임
        filter_column (str): 필터링할 열 이름 (예: '성별', '군구분')
        filter_value (str): 필터링할 값 (예: '남자', '육군')
    
    Returns:
        dict: 분석 결과를 포함한 딕셔너리
    """
    # 데이터 필터링
    if filter_column and filter_value:
        filtered_data = data[data[filter_column] == filter_value]
    else:
        filtered_data = data  # 필터링 기준이 없으면 전체 데이터 사용
    
    # 고유 사용자 수
    unique_user_count = count_unique_users(filtered_data)

    # 사전/사후 점수 추출
    pre_scores, post_scores = extract_pre_post_scores_from_csv(filtered_data)

    # 통계 분석 결과
    stats_result = calculate_statistics(pre_scores, post_scores)

    # 변화 분석 결과
    change_stats = calculate_change_statistics(pre_scores, post_scores)

    # 출력
    criteria_label = f"{filter_value} ({filter_column})" if filter_column else "전체 사용자"
    print(f"\n=== 분석 결과: {criteria_label} ===")
    print(f"총 사용자 수 (중복 제거): {unique_user_count}")
    print(f"사전 검사 평균: {stats_result['pre_mean']:.2f}, 표준편차: {stats_result['pre_std']:.2f}, 표준오차평균: {stats_result['pre_sem']:.2f}")
    print(f"사후 검사 평균: {stats_result['post_mean']:.2f}, 표준편차: {stats_result['post_std']:.2f}, 표준오차평균: {stats_result['post_sem']:.2f}")
    print(f"변화 평균: {stats_result['change_mean']:.2f}, 표준편차: {stats_result['change_std']:.2f}")
    print(f"95% 신뢰구간: [{stats_result['confidence_interval'][0]:.2f}, {stats_result['confidence_interval'][1]:.2f}]")
    print(f"t-value: {stats_result['t_value']:.2f}, p-value: {stats_result['p_value']:.4f}, df: {stats_result['df']}")
    
    print("\n점수 변화 분포:")
    for category, stats in change_stats.items():
        label = {"increase": "증가", "decrease": "감소", "no_change": "변화없음"}.get(category, category)
        print(f"{label} - 인원 (명): {stats['count']}, 평균: {stats['mean']}, 표준편차: {stats['std']}, 표준오차: {stats['sem']}")
    
    # 결과 반환
    return {
        "unique_user_count": unique_user_count,
        "statistics": stats_result,
        "change_distribution": change_stats
    }


* 전체 사용자

In [9]:
overall_result = analyze_by_criteria(raw_contents_info)



=== 분석 결과: 전체 사용자 ===
총 사용자 수 (중복 제거): 71
사전 검사 평균: 3.22, 표준편차: 3.20, 표준오차평균: 0.29
사후 검사 평균: 3.36, 표준편차: 3.24, 표준오차평균: 0.29
변화 평균: 0.14, 표준편차: 2.56
95% 신뢰구간: [-0.32, 0.60]
t-value: -0.60, p-value: 0.5472, df: 120

점수 변화 분포:
증가 - 인원 (명): 44, 평균: 2.5454545454545454, 표준편차: 1.9223840026643453, 표준오차: 0.28981029271905495
감소 - 인원 (명): 35, 평균: -2.7142857142857144, 표준편차: 1.6009450990224596, 표준오차: 0.2706091124051196
변화없음 - 인원 (명): 42, 평균: 0.0, 표준편차: 0.0, 표준오차: 0.0


* 성별

In [10]:
male_result = analyze_by_criteria(raw_contents_info, filter_column="성별", filter_value="남자")
female_result = analyze_by_criteria(raw_contents_info, filter_column="성별", filter_value="여자")



=== 분석 결과: 남자 (성별) ===
총 사용자 수 (중복 제거): 65
사전 검사 평균: 3.18, 표준편차: 3.21, 표준오차평균: 0.30
사후 검사 평균: 3.34, 표준편차: 3.21, 표준오차평균: 0.30
변화 평균: 0.17, 표준편차: 2.59
95% 신뢰구간: [-0.31, 0.65]
t-value: -0.69, p-value: 0.4934, df: 113

점수 변화 분포:
증가 - 인원 (명): 42, 평균: 2.5714285714285716, 표준편차: 1.964847876478062, 표준오차: 0.3031826095064675
감소 - 인원 (명): 33, 평균: -2.696969696969697, 표준편차: 1.6295100583620312, 표준오차: 0.2836612913158948
변화없음 - 인원 (명): 39, 평균: 0.0, 표준편차: 0.0, 표준오차: 0.0

=== 분석 결과: 여자 (성별) ===
총 사용자 수 (중복 제거): 6
사전 검사 평균: 4.00, 표준편차: 3.11, 표준오차평균: 1.18
사후 검사 평균: 3.71, 표준편차: 3.95, 표준오차평균: 1.49
변화 평균: -0.29, 표준편차: 2.14
95% 신뢰구간: [-2.26, 1.69]
t-value: 0.35, p-value: 0.7358, df: 6

점수 변화 분포:
증가 - 인원 (명): 2, 평균: 2.0, 표준편차: 0.0, 표준오차: 0.0
감소 - 인원 (명): 2, 평균: -3.0, 표준편차: 1.4142135623730951, 표준오차: 1.0
변화없음 - 인원 (명): 3, 평균: 0.0, 표준편차: 0.0, 표준오차: 0.0


* 군소속

In [11]:
# 군소속별 분석
services = ["육군", "공군", "해군", "해병대", "국직", "국방부"]

results_by_service = {}
for service in services:
    print(f"\n=== {service} 분석 ===")
    results_by_service[service] = analyze_by_criteria(raw_contents_info, filter_column="군구분", filter_value=service)



=== 육군 분석 ===

=== 분석 결과: 육군 (군구분) ===
총 사용자 수 (중복 제거): 61
사전 검사 평균: 3.18, 표준편차: 3.28, 표준오차평균: 0.32
사후 검사 평균: 3.35, 표준편차: 3.30, 표준오차평균: 0.32
변화 평균: 0.17, 표준편차: 2.50
95% 신뢰구간: [-0.31, 0.66]
t-value: -0.70, p-value: 0.4842, df: 104

점수 변화 분포:
증가 - 인원 (명): 37, 평균: 2.5675675675675675, 표준편차: 2.0485840711810432, 표준오차: 0.3367851467120495
감소 - 인원 (명): 30, 평균: -2.566666666666667, 표준편차: 1.4546793303071945, 표준오차: 0.2655868943819196
변화없음 - 인원 (명): 38, 평균: 0.0, 표준편차: 0.0, 표준오차: 0.0

=== 공군 분석 ===

=== 분석 결과: 공군 (군구분) ===
총 사용자 수 (중복 제거): 4
사전 검사 평균: 4.71, 표준편차: 3.25, 표준오차평균: 1.23
사후 검사 평균: 4.14, 표준편차: 2.91, 표준오차평균: 1.10
변화 평균: -0.57, 표준편차: 3.74
95% 신뢰구간: [-4.03, 2.88]
t-value: 0.40, p-value: 0.6997, df: 6

점수 변화 분포:
증가 - 인원 (명): 3, 평균: 2.3333333333333335, 표준편차: 1.5275252316519468, 표준오차: 0.881917103688197
감소 - 인원 (명): 2, 평균: -5.5, 표준편차: 2.1213203435596424, 표준오차: 1.4999999999999998
변화없음 - 인원 (명): 2, 평균: 0.0, 표준편차: 0.0, 표준오차: 0.0

=== 해군 분석 ===

=== 분석 결과: 해군 (군구분) ===
총 사용자 수 (중복 제거): 3
사전 검사 평균: 1.

c:\Users\ppjw0\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\ppjw0\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\ppjw0\AppData\Local\Temp\ipykernel_2944\3340829375.py:27: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  "sem": sem(group)
c:\Users\ppjw0\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\ppjw0\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = 

* 장병/간부

In [12]:
# 장병/간부에 따른 분석
roles = ["장병", "간부"]

results_by_role = {}
for role in roles:
    print(f"\n=== {role} 분석 ===")
    results_by_role[role] = analyze_by_criteria(raw_contents_info, filter_column="구분", filter_value=role)



=== 장병 분석 ===

=== 분석 결과: 장병 (구분) ===
총 사용자 수 (중복 제거): 49
사전 검사 평균: 2.93, 표준편차: 2.96, 표준오차평균: 0.32
사후 검사 평균: 3.28, 표준편차: 3.11, 표준오차평균: 0.33
변화 평균: 0.35, 표준편차: 2.59
95% 신뢰구간: [-0.20, 0.90]
t-value: -1.28, p-value: 0.2056, df: 87

점수 변화 분포:
증가 - 인원 (명): 36, 평균: 2.5833333333333335, 표준편차: 2.0336455654106214, 표준오차: 0.3389409275684369
감소 - 인원 (명): 24, 평균: -2.5833333333333335, 표준편차: 1.4719601443879746, 표준오차: 0.30046260628866583
변화없음 - 인원 (명): 28, 평균: 0.0, 표준편차: 0.0, 표준오차: 0.0

=== 간부 분석 ===

=== 분석 결과: 간부 (구분) ===
총 사용자 수 (중복 제거): 22
사전 검사 평균: 4.00, 표준편차: 3.71, 표준오차평균: 0.65
사후 검사 평균: 3.58, 표준편차: 3.61, 표준오차평균: 0.63
변화 평균: -0.42, 표준편차: 2.42
95% 신뢰구간: [-1.28, 0.44]
t-value: 1.01, p-value: 0.3223, df: 32

점수 변화 분포:
증가 - 인원 (명): 8, 평균: 2.375, 표준편차: 1.407885953173359, 표준오차: 0.4977628523130841
감소 - 인원 (명): 11, 평균: -3.0, 표준편차: 1.8973665961010275, 표준오차: 0.5720775535473553
변화없음 - 인원 (명): 14, 평균: 0.0, 표준편차: 0.0, 표준오차: 0.0


* 입대(임관년도)

In [13]:
# 분류 기준 설정
year_categories = {
    "2020년 이전": (raw_contents_info["임관년도"] < 2021),
    "2020~2022년": (raw_contents_info["임관년도"] >= 2021) & (raw_contents_info["임관년도"] <= 2022),
    "2023년 이후": (raw_contents_info["임관년도"] > 2022)
}
# 입대(임관)년도별 분석
results_by_year = {}
for label, condition in year_categories.items():
    print(f"\n=== {label} 분석 ===")
    filtered_data = raw_contents_info[condition]
    results_by_year[label] = analyze_by_criteria(filtered_data)


=== 2020년 이전 분석 ===

=== 분석 결과: 전체 사용자 ===
총 사용자 수 (중복 제거): 14
사전 검사 평균: 4.30, 표준편차: 3.72, 표준오차평균: 0.78
사후 검사 평균: 4.39, 표준편차: 3.94, 표준오차평균: 0.82
변화 평균: 0.09, 표준편차: 1.81
95% 신뢰구간: [-0.69, 0.87]
t-value: -0.23, p-value: 0.8196, df: 22

점수 변화 분포:
증가 - 인원 (명): 6, 평균: 2.3333333333333335, 표준편차: 1.0327955589886444, 표준오차: 0.4216370213557839
감소 - 인원 (명): 5, 평균: -2.4, 표준편차: 1.140175425099138, 표준오차: 0.5099019513592785
변화없음 - 인원 (명): 12, 평균: 0.0, 표준편차: 0.0, 표준오차: 0.0

=== 2020~2022년 분석 ===

=== 분석 결과: 전체 사용자 ===
총 사용자 수 (중복 제거): 22
사전 검사 평균: 2.60, 표준편차: 3.04, 표준오차평균: 0.55
사후 검사 평균: 2.60, 표준편차: 2.54, 표준오차평균: 0.46
변화 평균: 0.00, 표준편차: 3.06
95% 신뢰구간: [-1.14, 1.14]
t-value: 0.00, p-value: 1.0000, df: 29

점수 변화 분포:
증가 - 인원 (명): 12, 평균: 2.8333333333333335, 표준편차: 1.696699112626596, 표준오차: 0.4897948447043822
감소 - 인원 (명): 11, 평균: -3.090909090909091, 표준편차: 1.9725387425622571, 표준오차: 0.5947428085016773
변화없음 - 인원 (명): 7, 평균: 0.0, 표준편차: 0.0, 표준오차: 0.0

=== 2023년 이후 분석 ===

=== 분석 결과: 전체 사용자 ===
총 사용자 수 (중복 제거): 3

* 출생년도

In [14]:
# 출생년도 분류 기준 설정
birth_year_categories = {
    "1995년 이전": (raw_contents_info["생년월일"] < 1995),
    "1995~2001년": (raw_contents_info["생년월일"] >= 1995) & (raw_contents_info["생년월일"] <= 2001),
    "2002년 이후": (raw_contents_info["생년월일"] > 2001)
}
# 출생년도별 분석
results_by_birth_year = {}
for label, condition in birth_year_categories.items():
    print(f"\n=== {label} 분석 ===")
    filtered_data = raw_contents_info[condition]
    results_by_birth_year[label] = analyze_by_criteria(filtered_data)


=== 1995년 이전 분석 ===

=== 분석 결과: 전체 사용자 ===
총 사용자 수 (중복 제거): 21
사전 검사 평균: 4.09, 표준편차: 3.64, 표준오차평균: 0.62
사후 검사 평균: 3.82, 표준편차: 3.77, 표준오차평균: 0.65
변화 평균: -0.26, 표준편차: 2.30
95% 신뢰구간: [-1.07, 0.54]
t-value: 0.67, p-value: 0.5069, df: 33

점수 변화 분포:
증가 - 인원 (명): 9, 평균: 2.3333333333333335, 표준편차: 1.3228756555322954, 표준오차: 0.44095855184409843
감소 - 인원 (명): 11, 평균: -2.727272727272727, 표준편차: 1.793929156399945, 표준오차: 0.5408899920234007
변화없음 - 인원 (명): 14, 평균: 0.0, 표준편차: 0.0, 표준오차: 0.0

=== 1995~2001년 분석 ===

=== 분석 결과: 전체 사용자 ===
총 사용자 수 (중복 제거): 21
사전 검사 평균: 2.59, 표준편차: 2.93, 표준오차평균: 0.54
사후 검사 평균: 2.41, 표준편차: 2.43, 표준오차평균: 0.45
변화 평균: -0.17, 표준편차: 2.51
95% 신뢰구간: [-1.13, 0.78]
t-value: 0.37, p-value: 0.7140, df: 28

점수 변화 분포:
증가 - 인원 (명): 10, 평균: 2.3, 표준편차: 1.1595018087284057, 표준오차: 0.36666666666666664
감소 - 인원 (명): 9, 평균: -3.111111111111111, 표준편차: 1.7638342073763937, 표준오차: 0.5879447357921312
변화없음 - 인원 (명): 10, 평균: 0.0, 표준편차: 0.0, 표준오차: 0.0

=== 2002년 이후 분석 ===

=== 분석 결과: 전체 사용자 ===
총 사용자 수 (중복 제거

앱 사용 분포
---

* 콘텐츠별 이용 정보 함수

In [15]:
def analyze_content_usage(data):
    """
    콘텐츠별 및 전체 콘텐츠 이용 정보를 분석합니다.
    """
    content_user_stats = defaultdict(lambda: defaultdict(int))
    
    # 데이터 순회
    for _, row in data.iterrows():
        try:
            content_logs = json.loads(row["DATA"])
            user_id = row["사용자ID"]
            for log in content_logs:
                content_name = log.get("CONTENTS_NAME")
                if content_name:
                    content_user_stats[content_name][user_id] += 1
        except (KeyError, ValueError, TypeError):
            continue

    # 콘텐츠별 통계 계산
    content_analysis = []
    total_users = data["사용자ID"].nunique()
    user_total_usage = defaultdict(int)  # 사용자별 모든 콘텐츠 사용 횟수 합산

    for content_name, user_usage in content_user_stats.items():
        usage_counts = list(user_usage.values())  # 사용자별 사용 횟수
        user_count = len(user_usage)
        total_usage = sum(usage_counts)
        

        # 사용자별 총합 업데이트
        for user_id, count in user_usage.items():
            user_total_usage[user_id] += count

        content_analysis.append({
            "콘텐츠 명": content_name,
            "이용 인원 (명)": user_count,
            "이용 인원 비율 (%)": round(user_count / total_users * 100, 2) if total_users > 0 else 0,
            "평균 (회)": round(total_usage / user_count, 2) if user_count > 0 else 0,
            "표준편차": round(pd.Series(usage_counts).std(), 2) if user_count > 1 else 0,
            "최솟값 (회)": min(usage_counts) if usage_counts else 0,
            "최댓값 (회)": max(usage_counts) if usage_counts else 0,
        })

    # 전체 콘텐츠 추가
    overall_stats = {
        "콘텐츠 명": "전체 콘텐츠",
        "이용 인원 (명)": total_users,
        "이용 인원 비율 (%)": 100.0,
        "평균 (회)": round(sum([stat["평균 (회)"] * stat["이용 인원 (명)"] for stat in content_analysis]) / total_users, 2),
        "표준편차": round(pd.Series(list(user_total_usage.values())).std(), 2) if len(user_total_usage) > 1 else 0,
        "최솟값 (회)": min(user_total_usage.values()) if user_total_usage else 0,
        "최댓값 (회)": max(user_total_usage.values()) if user_total_usage else 0,  # 사용자별 총합 최댓값 계산
    }

    content_df = pd.concat([pd.DataFrame([overall_stats]), pd.DataFrame(content_analysis)], ignore_index=True)
    
    # 정렬 순서를 정의
    sort_order = [
        "전체 콘텐츠",
        "emotion_diary",
        "finding_blue_boat",
        "finding_blue_fishing",
        "finding_blue_parachute",
        "mandala",
        "mindteaching_webtoon",
        "mindfulness",
        "mysound_current",
        "mysound_pursue",
        "mysound_listen",
    ]

    # 콘텐츠 명 컬럼을 Categorical로 설정하여 정렬
    content_df["콘텐츠 명"] = pd.Categorical(content_df["콘텐츠 명"], categories=sort_order, ordered=True)
    content_df = content_df.sort_values("콘텐츠 명").reset_index(drop=True)

    return content_df


In [16]:
def analyze_by_criteria(data, filter_column, filter_value):
    """
    특정 기준에 따라 데이터를 필터링하고 콘텐츠별 이용 정보를 분석합니다.

    Args:
        data (pd.DataFrame): 원본 데이터프레임
        filter_column (str): 필터링할 컬럼명
        filter_value (str): 필터링 조건 값

    Returns:
        pd.DataFrame: 필터링된 데이터에 대한 콘텐츠별 이용 정보
    """
    # 필터링된 데이터
    filtered_data = data[data[filter_column] == filter_value]

    # 분석 수행
    return analyze_content_usage(filtered_data)


* 전체사용자

In [17]:
content_analysis_df = analyze_content_usage(raw_contents_info)

# 결과 출력
print(content_analysis_df)

                     콘텐츠 명  이용 인원 (명)  이용 인원 비율 (%)  평균 (회)   표준편차  최솟값 (회)  \
0                   전체 콘텐츠         71        100.00   37.87  48.05        2   
1            emotion_diary         67         94.37   19.03  21.93        1   
2        finding_blue_boat         19         26.76    2.68   3.18        1   
3     finding_blue_fishing         25         35.21    2.56   3.00        1   
4   finding_blue_parachute         23         32.39    3.35   3.28        1   
5                  mandala         12         16.90    2.92   3.34        1   
6     mindteaching_webtoon         43         60.56   19.28  24.32        1   
7              mindfulness         44         61.97    6.61   9.02        1   
8          mysound_current         12         16.90    2.17   2.59        1   
9           mysound_pursue          6          8.45    3.50   4.28        1   
10          mysound_listen          4          5.63    5.00   7.35        1   

    최댓값 (회)  
0       291  
1       133  
2        

* 성별

In [22]:
# 남성에 대한 결과
male_result = analyze_by_criteria(raw_contents_info, filter_column="성별", filter_value="남자")

# 여성에 대한 결과
female_result = analyze_by_criteria(raw_contents_info, filter_column="성별", filter_value="여자")

# 결과 출력
print("남성 결과:")
print(male_result)

print("\n여성 결과:")
print(female_result)


남성 결과:
                     콘텐츠 명  이용 인원 (명)  이용 인원 비율 (%)  평균 (회)   표준편차  최솟값 (회)  \
0                   전체 콘텐츠         65        100.00   40.41  49.43        2   
1            emotion_diary         62         95.38   19.82  22.54        1   
2        finding_blue_boat         18         27.69    2.72   3.27        1   
3     finding_blue_fishing         23         35.38    2.70   3.10        1   
4   finding_blue_parachute         21         32.31    3.57   3.36        1   
5                  mandala         10         15.38    3.30   3.56        1   
6     mindteaching_webtoon         43         66.15   19.28  24.32        1   
7              mindfulness         41         63.08    7.02   9.22        1   
8          mysound_current         11         16.92    2.27   2.69        1   
9           mysound_pursue          5          7.69    3.60   4.77        1   
10          mysound_listen          3          4.62    6.33   8.39        1   

    최댓값 (회)  
0       291  
1       133  
2 

* 군소속

In [27]:
# 군 소속별 분석
army_result = analyze_by_criteria(raw_contents_info, filter_column="군구분", filter_value="육군")
navy_result = analyze_by_criteria(raw_contents_info, filter_column="군구분", filter_value="해군")
airforce_result = analyze_by_criteria(raw_contents_info, filter_column="군구분", filter_value="공군")
sea_result = analyze_by_criteria(raw_contents_info, filter_column="군구분", filter_value="해병대")
korea_result = analyze_by_criteria(raw_contents_info, filter_column="군구분", filter_value="국방부")

# 결과 출력
print("육군 결과:")
print(army_result)

print("\n해군 결과:")
print(navy_result)

print("\n공군 결과:")
print(airforce_result)

print("\n해병대 결과:")
print(sea_result)

print("\n국방부 결과:")
print(korea_result)


육군 결과:
                     콘텐츠 명  이용 인원 (명)  이용 인원 비율 (%)  평균 (회)   표준편차  최솟값 (회)  \
0                   전체 콘텐츠         61        100.00   40.47  50.89        2   
1            emotion_diary         58         95.08   19.60  22.85        1   
2        finding_blue_boat         17         27.87    2.35   2.96        1   
3     finding_blue_fishing         22         36.07    2.64   3.16        1   
4   finding_blue_parachute         20         32.79    3.00   3.11        1   
5                  mandala         10         16.39    3.30   3.56        1   
6     mindteaching_webtoon         41         67.21   19.80  24.80        1   
7              mindfulness         36         59.02    7.36   9.60        1   
8          mysound_current         11         18.03    2.18   2.71        1   
9           mysound_pursue          6          9.84    3.50   4.28        1   
10          mysound_listen          3          4.92    6.33   8.39        1   

    최댓값 (회)  
0       291  
1       133  
2 

* 장병/간부

In [29]:
# 장병과 간부별 분석
soldier_result = analyze_by_criteria(raw_contents_info, filter_column="구분", filter_value="장병")
officer_result = analyze_by_criteria(raw_contents_info, filter_column="구분", filter_value="간부")

# 결과 출력
print("장병 결과:")
print(soldier_result)

print("\n간부 결과:")
print(officer_result)


장병 결과:
                     콘텐츠 명  이용 인원 (명)  이용 인원 비율 (%)  평균 (회)   표준편차  최솟값 (회)  \
0                   전체 콘텐츠         49        100.00   40.04  49.29        2   
1            emotion_diary         48         97.96   19.62  23.81        1   
2        finding_blue_boat         12         24.49    2.67   3.03        1   
3     finding_blue_fishing         14         28.57    2.79   2.58        1   
4   finding_blue_parachute         14         28.57    3.36   2.73        1   
5                  mandala          7         14.29    1.71   1.50        1   
6     mindteaching_webtoon         35         71.43   17.86  23.34        1   
7              mindfulness         34         69.39    6.68   8.98        1   
8          mysound_current          8         16.33    1.62   0.92        1   
9           mysound_pursue          4          8.16    1.50   1.00        1   
10          mysound_listen          3          6.12    6.33   8.39        1   

    최댓값 (회)  
0       291  
1       133  
2 

* 입대년도

In [30]:
# 분류 기준 설정
year_categories = {
    "2023년 이전": raw_contents_info["임관년도"] < 2023,
    "2023년 이후": raw_contents_info["임관년도"] >= 2023
}

# 분석 결과 저장 딕셔너리
year_results = {}

# 각 분류 기준에 대해 분석
for category, condition in year_categories.items():
    filtered_data = raw_contents_info[condition]
    year_results[category] = analyze_content_usage(filtered_data)

# 결과 출력
for category, result in year_results.items():
    print(f"\n[{category}] 결과:")
    print(result)



[2023년 이전] 결과:
                     콘텐츠 명  이용 인원 (명)  이용 인원 비율 (%)  평균 (회)   표준편차  최솟값 (회)  \
0                   전체 콘텐츠         36        100.00   29.33  35.66        2   
1            emotion_diary         33         91.67   15.03  15.16        1   
2        finding_blue_boat          9         25.00    2.56   3.24        1   
3     finding_blue_fishing         15         41.67    2.73   3.53        1   
4   finding_blue_parachute         13         36.11    3.77   3.90        1   
5                  mandala          8         22.22    3.25   3.96        1   
6     mindteaching_webtoon         19         52.78   15.05  18.98        1   
7              mindfulness         20         55.56    4.60   5.32        1   
8          mysound_current          8         22.22    2.50   3.12        1   
9           mysound_pursue          4         11.11    4.75   4.92        1   
10          mysound_listen          3          8.33    1.33   0.58        1   

    최댓값 (회)  
0       163  
1      

* 출생년도

In [33]:
# "생년월일" 기준으로 데이터 분류
birth_date_categories = {
    "2000년 이전": raw_contents_info["생년월일"] < 2000,
    "2000년 이후": raw_contents_info["생년월일"] >= 2000
}

# 분석 결과 저장 딕셔너리
birth_date_results = {}

# 각 분류 기준에 대해 분석
for category, condition in birth_date_categories.items():
    filtered_data = raw_contents_info[condition]
    birth_date_results[category] = analyze_content_usage(filtered_data)

# 결과 출력
for category, result in birth_date_results.items():
    print(f"\n[{category}] 결과:")
    print(result)




[2000년 이전] 결과:
                     콘텐츠 명  이용 인원 (명)  이용 인원 비율 (%)  평균 (회)   표준편차  최솟값 (회)  \
0                   전체 콘텐츠         32        100.00   29.53  38.70        2   
1            emotion_diary         29         90.62   16.00  14.49        1   
2        finding_blue_boat          9         28.12    2.44   3.24        1   
3     finding_blue_fishing         14         43.75    2.29   3.20        1   
4   finding_blue_parachute         11         34.38    3.73   4.15        1   
5                  mandala          7         21.88    3.57   4.16        1   
6     mindteaching_webtoon         15         46.88   17.33  22.80        1   
7              mindfulness         15         46.88    4.87   8.05        1   
8          mysound_current          5         15.62    3.00   3.94        1   
9           mysound_pursue          1          3.12   12.00   0.00       12   
10          mysound_listen          1          3.12    1.00   0.00        1   

    최댓값 (회)  
0       163  
1      

콘텐츠와 우울 척도 점수 간의 상관관계
--

* 회귀 분석 함수

In [54]:
import math
from collections import defaultdict

In [73]:
# 회귀 분석을 위한 콘텐츠별 feature 값 처리 함수들

def parse_dates(content):
    start_time = datetime.strptime(str(content["START_DATE"]), "%Y-%m-%d %H:%M:%S")
    end_time = datetime.strptime(str(content["END_DATE"]), "%Y-%m-%d %H:%M:%S")
    return start_time, end_time

def calculate_averages(lists):
    return [
        sum(item for item in lst if isinstance(item, (int, float))) / len(lst) if len(lst) > 0 else 0
        for lst in lists
    ]

def process_content(data, content_name, process_function):
    results = []
    total_usage_time = 0
    for content in data["DATA"]:
        if content:
            if content['CONTENTS_NAME'] == content_name:
                result = process_function(content)
                results.append(result[1:])  # Exclude usage_time from results
                total_usage_time += result[0]  # Sum usage_time
    return total_usage_time, list(map(list, zip(*results)))  # Transpose the list of results excluding usage_time

def process_content_per_group(data, content_name, process_function):
    results = []
    total_usage_time = 0
    for content_data in data:
        if content_data["DATA"]:
            for content in content_data["DATA"]:
                if content['CONTENTS_NAME'] == content_name:
                    result = process_function(content)
                    results.append(result[1:])  # Exclude usage_time from results
                    total_usage_time += result[0]  # Sum usage_time
    return total_usage_time, list(map(list, zip(*results)))  # Transpose the list of results excluding usage_time


def generic_process(content):
    start_time, end_time = parse_dates(content)
    usage_time = (end_time - start_time).total_seconds()
    return [usage_time]  # Return usage_time as the first element

def emotion_diary_process(content):
    result = generic_process(content)
    additional_fields = ["VALENCE", "AROUSAL", "HAPPY", "JOY", "CALM", "NEUTRAL", "DEPRESSED", "SAD", "ANGRY", "SUICIDE_LEVEL"]
    return result + [content[field] for field in additional_fields]

def findingblue_process(content, content_name, score_name):
    result = generic_process(content)
    score = content[score_name] if math.isnan(content[score_name]) is False else 0
    reaction_times = []
    correct_reactions, incorrect_reactions = 0, 0
    for raw in content["RAW"]:
        if raw["Reaction_Correct"] == "Y":
            reaction_time = raw["TimeStamp_ms"] if content_name == 'finding_blue_fishing' else raw["Reaction_Time_ms"]
            reaction_times.append(reaction_time)
            correct_reactions += 1
        else:
            incorrect_reactions += 1
    correct_reaction_ratio = correct_reactions / (correct_reactions + incorrect_reactions) if (correct_reactions + incorrect_reactions) > 0 else 0
    return result + [score, sum(reaction_times) / len(reaction_times) if len(reaction_times) > 0 else 0, correct_reaction_ratio]

def mandala_process(content):
    result = generic_process(content)
    additional_fields = ["SELECTED_MANDALA_NUMBER", "SELECTED_MUSIC_NUMBER", "SELECTED_FEELING", "FIRST_COLOR_TIME_MS", "FINISH_COLOR_TIME_MS", "DEGREE_OF_COMPLETION", "MODIFY_COUNT", "DIVERSITY_COLOR"]
    return result + [content[field] for field in additional_fields]

def mindfulness_process(content):
    result = generic_process(content)
    additional_fields = ["PRE_EMOTION", "SESSION", "PROGRESS", "POST_EMOTION"]
    processed_fields = [content[field] if field != "PROGRESS" or math.isnan(content["PROGRESS"]) is False else 0 for field in additional_fields]
    return result + processed_fields

def mysound_process(content):
    result = generic_process(content)
    additional_fields = ["PRE_EMOTION", "SELECT_BEAT", "N_EACH_BEAT_PLAYED", "SELECTED_INSTRUMENT_LIST", "N_EACH_INSTRUMENT_PLAYED", "POST_EMOTION"]
    processed_fields = [content[field] if field not in ["N_EACH_BEAT_PLAYED", "SELECTED_INSTRUMENT_LIST", "N_EACH_INSTRUMENT_PLAYED"] else sum(content[field]) for field in additional_fields]
    return result + processed_fields

def mysound_listen_process(content):
    result = generic_process(content)
    list_played_song_num = len(content["PLAY_LIST"])
    return result + [list_played_song_num]

def compute_means(data):
    results = defaultdict(list)
    content_process_map = {
        'emotion_diary': emotion_diary_process,
        'finding_blue_boat': lambda content: findingblue_process(content, 'finding_blue_boat', 'CONGRUENCY_SCORE'),
        'finding_blue_fishing': lambda content: findingblue_process(content, 'finding_blue_fishing', 'SOCIAL_SCORE'),
        'finding_blue_parachute': lambda content: findingblue_process(content, 'finding_blue_parachute', 'PARA_SCORE'),
        'mandala': mandala_process,
        'mindfulness': mindfulness_process,
        'mindteaching_webtoon': generic_process,
        'mysound_current': mysound_process,
        'mysound_pursue': mysound_process,
        'mysound_listen': mysound_listen_process
    }

    for key, process_function in content_process_map.items():
        total_usage_time, processed_data = process_content(data, key, process_function)
        if processed_data:
            averages = calculate_averages(processed_data)
            results[key].append([total_usage_time] + averages)
        else:
            results[key].append([total_usage_time])

    means = {key: np.mean(vals, axis=0).tolist() for key, vals in results.items()}
    return means

def compute_means_per_group(datas):
    results = defaultdict(lambda: defaultdict(list))
    content_process_map = {
        'emotion_diary': emotion_diary_process,
        'finding_blue_boat': lambda content: findingblue_process(content, 'finding_blue_boat', 'CONGRUENCY_SCORE'),
        'finding_blue_fishing': lambda content: findingblue_process(content, 'finding_blue_fishing', 'SOCIAL_SCORE'),
        'finding_blue_parachute': lambda content: findingblue_process(content, 'finding_blue_parachute', 'PARA_SCORE'),
        'mandala': mandala_process,
        'mindfulness': mindfulness_process,
        'mindteaching_webtoon': generic_process,
        'mysound_current': mysound_process,
        'mysound_pursue': mysound_process,
        'mysound_listen': mysound_listen_process
    }

    for index, data in enumerate(datas):
        for key, process_function in content_process_map.items():
            total_usage_time, processed_data = process_content_per_group(data, key, process_function)
            if processed_data:
                averages = calculate_averages(processed_data)
                results[index][key].append([total_usage_time] + averages)
            else:
                results[index][key].append([total_usage_time])

    means = {index: {key: np.mean(vals, axis=0).tolist() for key, vals in inner_dict.items()} for index, inner_dict in results.items()}
    return means

In [ ]:
# 1. 데이터 전처리
def preprocess_data(data):
    """
    DATA 열의 JSON 데이터를 파싱하여 필요한 정보를 추출합니다.
    """
    rows = []
    for _, row in data.iterrows():
        try:
            content_logs = json.loads(row["DATA"])
            for log in content_logs:
                log["사용자ID"] = row["사용자ID"]  # 사용자 ID 추가
                rows.append(log)
        except (KeyError, ValueError, TypeError):
            continue

    return pd.DataFrame(rows)

# 2. 피처 처리
def parse_dates(content):
    """
    시작 날짜와 종료 날짜를 파싱하여 사용 시간을 계산합니다.
    """
    try:
        start_time = datetime.strptime(content["START_DATE"], "%Y-%m-%dT%H:%M:%S")
        end_time = datetime.strptime(content["END_DATE"], "%Y-%m-%dT%H:%M:%S")
        usage_time = (end_time - start_time).total_seconds()
    except (ValueError, TypeError):
        usage_time = 0
    return usage_time

def process_content(content, fields):
    """
    특정 콘텐츠의 사용 시간과 추가 필드를 추출합니다.
    """
    usage_time = parse_dates(content)
    processed_fields = [content.get(field, 0) for field in fields]
    return [usage_time] + processed_fields

# 3. 콘텐츠별 데이터 처리
def extract_features(preprocessed_data, content_name, fields):
    """
    콘텐츠별로 데이터를 처리하여 피처를 추출합니다.
    """
    # 해당 콘텐츠 이름에 해당하는 데이터 필터링
    content_data = preprocessed_data[preprocessed_data["CONTENTS_NAME"] == content_name]
    
    features = []
    for _, content in content_data.iterrows():
        # 각 콘텐츠의 사용 시간과 추가 필드 처리
        processed = process_content(content, fields)
        # POST_SCORE 추가
        score = content.get("POST_SCORE", 0)  # "POST_SCORE"가 없으면 기본값 0
        processed.append(score)
        features.append(processed)

    # "POST_SCORE"를 "score"로 포함한 컬럼 정의
    return pd.DataFrame(features, columns=["USAGE_TIME"] + fields + ["score"])

# 4. 회귀 분석
def perform_regression(features, features_name):
    """
    회귀 분석을 수행합니다.
    """
    regression_results = {}
    for content_name, feature_list in features_name.items():
        features = extract_features(preprocessed_data, content_name, feature_list)
        
        # 종속 변수: score로 가정
        X = features.drop(columns=["score"])
        y = features["score"]
        
        # 결측값 및 숫자형 변환 처리
        X = X.replace([np.inf, -np.inf], np.nan).dropna()
        y = y[X.index]
        X = X.apply(pd.to_numeric, errors="coerce")  # 숫자로 변환, 변환 불가한 값은 NaN 처리
        y = pd.to_numeric(y, errors="coerce")
        
        # NaN 제거
        X = X.dropna()
        y = y.loc[X.index]

        # 상수항 추가 및 회귀 수행
        X = sm.add_constant(X)
        model = sm.OLS(y, X).fit()
        regression_results[content_name] = model

    return regression_results


# 5. 콘텐츠 정의
features_name = {
    "emotion_diary": ["USAGE_TIME", "USAGE_NUM", "VALENCE", "AROUSAL", "HAPPY", "JOY", "CALM", "NEUTRAL", "DEPRESSED", "SAD", "ANGRY", "SUICIDE_LEVEL"],
    "finding_blue_boat": ["USAGE_TIME", "USAGE_NUM", "SCORE", "REACTION_TIME", "CORRECT_REACTION_RATIO"],
    "finding_blue_fishing": ["USAGE_TIME", "USAGE_NUM", "SCORE", "REACTION_TIME", "CORRECT_REACTION_RATIO"], 
    "finding_blue_parachute": ["USAGE_TIME", "USAGE_NUM", "SCORE", "REACTION_TIME", "CORRECT_REACTION_RATIO"],
    "mandala": ["USAGE_TIME", "USAGE_NUM", "SELECTED_MANDALA_NUMBER", "SELECTED_MUSIC_NUMBER", "SELECTED_FEELING", "FIRST_COLOR_TIME_MS", "FINISH_COLOR_TIME_MS", "DEGREE_OF_COMPLETION", "MODIFY_COUNT", "DIVERSITY_COLOR"], 
    "mindfulness": ["USAGE_TIME", "USAGE_NUM", "PRE_EMOTION", "SESSION", "PROGRESS", "POST_EMOTION"],
    "mindteaching_webtoon": ["USAGE_TIME", "USAGE_NUM"],
    "mysound_current": ["USAGE_TIME", "USAGE_NUM", "PRE_EMOTION", "SELECT_BEAT", "N_EACH_BEAT_PLAYED", "SELECTED_INSTRUMENT_LIST", "N_EACH_INSTRUMENT_PLAYED", "POST_EMOTION"], 
    "mysound_pursue": ["USAGE_TIME", "USAGE_NUM", "PRE_EMOTION", "SELECT_BEAT", "N_EACH_BEAT_PLAYED", "SELECTED_INSTRUMENT_LIST", "N_EACH_INSTRUMENT_PLAYED", "POST_EMOTION"], 
    "mysound_listen": ["USAGE_TIME", "USAGE_NUM", "PLAY_LIST"]
}

# 데이터 로드 및 전처리
preprocessed_data = preprocess_data(raw_contents_info)


In [91]:
# 6. 회귀 분석 실행
regression_results = perform_regression(preprocessed_data, features_name)



TypeError: perform_regression() takes 1 positional argument but 2 were given

In [ ]:
# 7. 결과 출력
for content_name, model in regression_results.items():
    print(f"[{content_name}] R_squared: {model.rsquared:.3f}")
    print(model.summary())


ValueError: zero-size array to reduction operation maximum which has no identity